In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
train = pd.read_csv('datasets/hwu/train.csv')
val = pd.read_csv('datasets/hwu/val.csv')
test = pd.read_csv('datasets/hwu/test.csv')

In [ ]:
train

,text,category
0,what alarms do i have set right now,alarm_query
1,checkout today alarm of meeting,alarm_query
2,report alarm settings,alarm_query
3,see see for me the alarms that you have set to...,alarm_query
4,is there an alarm for ten am,alarm_query
...,...,...
8949,how hot is it in miami,weather_query
8950,will it snow next week,weather_query
8951,am i gonna need rain boots,weather_query
8952,should i bring warm clothes,weather_query


In [3]:
def tokenizer(text):
    tokens = text.split(' ')
    return tokens

In [4]:
train_tokens = train['text'].apply(tokenizer).to_list()
test_tokens = test['text'].apply(tokenizer).tolist()
print(len(train_tokens))
print(max(len(train_tokens[i]) for i in range(8954)))

8954
25


In [11]:
le = LabelEncoder()
train_labels = le.fit_transform(train['category'])
test_labels = le.transform(test['category'])
len(set(train_labels))

64

In [8]:
indexer = Tokenizer(oov_token="<OOV>")
indexer.fit_on_texts(train_tokens)
train_sequences = indexer.texts_to_sequences(train_tokens)
test_sequences = indexer.texts_to_sequences(test_tokens)
train_padded_seq = pad_sequences(train_sequences, maxlen=25, padding='post', truncating='post')
test_padded_seq = pad_sequences(test_sequences, maxlen=25, padding='post', truncating='post')

In [9]:
vocab_size = len(indexer.word_index) + 1
embedding_dim = 64

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    LSTM(units=64, return_sequences=False),
    Dense(64, activation='softmax'),
])

In [12]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(train_padded_seq, train_labels, epochs=10, validation_data=(test_padded_seq, test_labels), batch_size=32)

Epoch 1/10
280/280 [==============================] - 5s 13ms/step - loss: 3.9441 - accuracy: 0.0304 - val_loss: 3.5167 - val_accuracy: 0.0818
Epoch 2/10
280/280 [==============================] - 4s 13ms/step - loss: 2.9342 - accuracy: 0.1704 - val_loss: 2.5892 - val_accuracy: 0.2565
Epoch 3/10
280/280 [==============================] - 3s 11ms/step - loss: 2.2117 - accuracy: 0.3324 - val_loss: 2.2720 - val_accuracy: 0.3327
Epoch 4/10
280/280 [==============================] - 3s 10ms/step - loss: 1.7864 - accuracy: 0.4520 - val_loss: 1.9201 - val_accuracy: 0.4638
Epoch 5/10
280/280 [==============================] - 3s 10ms/step - loss: 1.4765 - accuracy: 0.5468 - val_loss: 1.7048 - val_accuracy: 0.5325
Epoch 6/10
280/280 [==============================] - 3s 10ms/step - loss: 1.2602 - accuracy: 0.6213 - val_loss: 1.6434 - val_accuracy: 0.5455
Epoch 7/10
280/280 [==============================] - 3s 10ms/step - loss: 1.0468 - accuracy: 0.6962 - val_loss: 1.6021 - val_accuracy: 0.5688